In [3]:
import os
%pwd

'c:\\Users\\SACHIN\\PycharmProjects\\Obesity-Risk-\\notebook'

In [4]:
os.chdir('../')
%pwd

'c:\\Users\\SACHIN\\PycharmProjects\\Obesity-Risk-'

In [5]:
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/ubalesachin22/Obesity-Risk-.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='ubalesachin22'
os.environ['MLFLOW_TRACKING_PASSWORD']='14c861b68786366b977abad8b824a891884d1216'

In [7]:
import os
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelEvaluationConfig:
    root: Path
    scaled_test_set: Path
    model: Path
    preprocessing_obj: Path
    metrics: Path
    all_params: dict
    mlflow_uri: str

In [8]:
from ObesityRisk.constants import *
from ObesityRisk.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationsManager:
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_file])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XgbClassifier
        
        create_directories([config.root])

        model_evaluation_config = ModelEvaluationConfig(
            root = config.root,
            scaled_test_set = config.scaled_test_set,
            model = config.model,
            preprocessing_obj= config.preprocessing_obj,
            metrics = config.metrics,
            all_params = params,
            mlflow_uri = 'https://dagshub.com/ubalesachin22/Obesity-Risk-.mlflow'
        )

        return model_evaluation_config

In [10]:
import os
import pandas as pd
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.metrics import accuracy_score
from src.ObesityRisk.utils.common import save_json
from src.ObesityRisk import logger
from src.ObesityRisk.utils.common import *

In [11]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config

    
    def model_evaluation(self, true, predicted):
        ac = accuracy_score(true, predicted)
        return ac

    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.scaled_test_set)

        logger.info("Slpitting Test data")
        test_x = test_data.iloc[:, :-1]
        test_y = test_data.iloc[:, -1]

        #test_y = test_data['NObeyesdad'].map(remap)

        logger.info('Loading Trained Model')
        model = load_pickle(self.config.model)


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            test_prediction = model.predict(test_x)

            ac = self.model_evaluation(test_y, test_prediction)

            scores = {"accuracy score": ac}
            save_json(path=Path(self.config.metrics), data = scores)
            
            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy score", ac)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, 'model', registered_model_name='XgbClassifier')
            else:
                mlflow.sklearn.log_model(model, 'model')


In [12]:
try:
    config = ConfigurationsManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-03-10 16:41:13,972: INFO: common: yaml file Config\Config.yaml loaded successfully]
[2024-03-10 16:41:13,994: INFO: common: yaml file params.yaml loaded successfully]
[2024-03-10 16:41:14,000: INFO: common: creating directory at artifacts]
[2024-03-10 16:41:14,006: INFO: common: creating directory at artifacts/model_evaluation]


[2024-03-10 16:41:14,107: INFO: 3416948005: Slpitting Test data]
[2024-03-10 16:41:14,116: INFO: 3416948005: Loading Trained Model]
[2024-03-10 16:41:18,266: INFO: common: jason file saved to as: artifacts\model_evaluation\metrics.json]


Registered model 'XgbClassifier' already exists. Creating a new version of this model...
2024/03/10 16:41:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XgbClassifier, version 14
Created version '14' of model 'XgbClassifier'.
